# Generate Data

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
from mlchecks import Dataset, validate_dataset, validate_dataset_or_dataframe
from mlchecks.checks.integrity.rare_format_detection import RareFormatDetection


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
np.random.seed(42)

# df = pd.DataFrame(np.random.choice(a=['BIG', 'STILL_BIG'], size=(200,3)), columns=['x1', 'x2', 'x3'])
# df = df.append({'x1': 'bla', 'x2': 'BIG', 'x3': 1}, ignore_index=True)
# df = df.append({'x1': 'bla', 'x2': 'BIG', 'x3': 1}, ignore_index=True)
# df = df.append({'x1': 'bla2', 'x2': 'BIG', 'x3': 2}, ignore_index=True)

In [4]:
datelist = pd.date_range(start=datetime.today(), periods=200, freq='D', normalize=True)
s_date = pd.Series([d.strftime('%Y-%m-%d') for d in datelist[:197]] + [d.strftime('%Y-%b-%d') for d in datelist[197:]], name='date')
emaillist = [''.join(np.random.choice(a=list('abcdefghijklmnopqrstuvwxyz'), p=[1/26]*26, size=np.random.choice(a=[6,7,8], p=[0.2, 0.5, 0.3]))) + '@gmail.com' for x in range(198)]
emaillist.extend(['myname@gmail.com1', 'myname@gmail.co'])
s_email = pd.Series(emaillist, name='email')

df = pd.DataFrame([s_date, s_email]).T

In [5]:
df

,date,email
0,2021-10-17,ytpeebw@gmail.com
1,2021-10-18,sazvfee@gmail.com
2,2021-10-19,nlhpdhj@gmail.com
3,2021-10-20,ufnpbpe@gmail.com
4,2021-10-21,yzvhcr@gmail.com
...,...,...
195,2022-04-30,zdsklss@gmail.com
196,2022-05-01,dcsphccx@gmail.com
197,2022-May-02,ifjbnb@gmail.com
198,2022-May-03,myname@gmail.com1


# Run Check

In [6]:
ds = Dataset(df)
check = RareFormatDetection()

check.run(dataset=ds)


c:\users\nirhu\documents\deepchecks\mlchecks\mlchecks\base\dataset.py:57: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._features = [x for x in df.columns if x not in {label, index, date}]
c:\users\nirhu\documents\deepchecks\mlchecks\mlchecks\base\dataset.py:67: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._cat_features = self.infer_categorical_features()


{'date':                                              digits only format (ignoring letters)  \
ratio of rare patterns to common patterns                                    1.52%   
common formats                                                        [0000-00-00]   
examples for values in common formats                                 [2021-10-18]   
values in rare formats                     [2022-May-02, 2022-May-03, 2022-May-04]   

                                          sequences of digits only format (ignoring letters)  \
ratio of rare patterns to common patterns                                              1.52%   
common formats                                                                  [____-__-__]   
examples for values in common formats                                           [2021-10-18]   
values in rare formats                               [2022-May-02, 2022-May-03, 2022-May-04]   

                                          sequences of letters only format (ignoring letters)  \
ratio of rare patterns to common patterns                                              1.52%    
common formats                                                                          [--]    
examples for values in common formats                                           [2021-10-18]    
values in rare formats                               [2022-May-02, 2022-May-03, 2022-May-04]    

                                                         digits and letters format  \
ratio of rare patterns to common patterns                                    1.52%   
common formats                                                        [0000-00-00]   
examples for values in common formats                                 [2021-10-18]   
values in rare formats                     [2022-May-02, 2022-May-03, 2022-May-04]   

                                          digits and letters format (case sensitive)  \
ratio of rare patterns to common patterns                                      1.52%   
common formats                                                          [0000-00-00]   
examples for values in common formats                                   [2021-10-18]   
values in rare formats                       [2022-May-02, 2022-May-03, 2022-May-04]   

                                                               any sequence format  
ratio of rare patterns to common patterns                                    1.52%  
common formats                                                        [0000-00-00]  
examples for values in common formats                                 [2021-10-18]  
values in rare formats                     [2022-May-02, 2022-May-03, 2022-May-04]  , 'email':                                           digits only format (ignoring letters)  \
ratio of rare patterns to common patterns                                 0.50%   
common formats                                                             [@.]   
examples for values in common formats                       [sazvfee@gmail.com]   
values in rare formats                                      [myname@gmail.com1]   

                                          sequences of digits only format (ignoring letters)  \
ratio of rare patterns to common patterns                                              0.50%   
common formats                                                                          [@.]   
examples for values in common formats                                    [sazvfee@gmail.com]   
values in rare formats                                                   [myname@gmail.com1]   

                                                                   digits and letters format  \
ratio of rare patterns to common patterns                                              1.01%   
common formats                             [XXXXXXX@XXXXX.XXX, XXXXXXXX@XXXXX.XXX, XXXXXX...   
examples for values in common formats      [sazvfee@gmail.com, qxmdstou@gmail.com, vssubj...   
value